<table style="border: none; border-collapse: collapse;" border="0" cellspacing="0" cellpadding="0" width="100%" align="center">
<tr style="border: none;">
<td  style="border: none;" width="20%">Brought to you by <h3>the dynamic trio</h3></td>
<td style="border: none;"><a href="https://github.com/fredokun/cl-jupyter"><img src="http://jupyter.org/assets/nav_logo.svg" alt="Project Jupyter" height="128"></a></td>
<td style="border: none;"><a href="http://clhs.lisp.se/"><img src="http://www.lisperati.com/lisplogo_fancy_128.png" alt="Public Domain Lisp Logo Set By Conrad Barski, M.D." height="128"></a></td><td style="border: none;"><a href="https://www.hdfgroup.org/"><img src="https://www.hdfgroup.org/images/hdf_logo.jpg" alt="The HDF Group" height="128"></a></td></tr>
</table>

# A Commentary on the HDF5 File Format

Obviously, no knowledge of the low-level byte layout is needed for the successful deployment of HDF5 data containers in your application or environment. Indeed, the number of HTTP requests directed at the [HDF5 File Format Specification](https://www.hdfgroup.org/HDF5/doc/H5.format.html) is a miniscule fraction of the requests directed at, for example, the [HDF5 Reference Manual](https://www.hdfgroup.org/HDF5/doc/RM/RM_H5Front.html). Both documents give almost opposite, but certainly complementary views of what we call an *HDF5 file*. From the perspective of the API, it remains somewhat mysterious how all those high-level abstractions (groups, extendible datasets, attributes, etc.) are ultimately arranged and manipulated in a linear byte stream. Conversely, how can this "digital DNA strand" reliably be translated back into recognizable objects?

The purpose of this sequence of articles is twofold: Firstly, I would like to convince you, dear reader, that the file format specifiction is a mystery-free and rather accessible document, which describes in unambiguous terms what's really in an HDF5 file. Secondly, I would like to help you appreciate what the HDF5 library does beyond merely implementing readers and writers conforming to the specfication. Just fishing around for a few bytes in an HDF5 file is what we do here, but a library it doth not make.

Since we have no ambition for a library, and this is a purely (data-)touristic endeavor, we choose an environment that lets us focus on the problem at hand, and that doesn't distract with concerns over representation, or noisy syntax.
I believe the combination of Jupyter notebooks with Common Lisp offers an entertaining and concise mixture, but I'll let you be the judge of that.

#### Logistics

A Docker container that includes all the dependencies to run the notebooks in this series is available for [download](https://hub.docker.com/r/hapax/hdf5-works/). It includes:
- A [Jupyter](http://jupyter.org/) notebook server
- The [Steel Bank Common Lisp](http://www.sbcl.org/) (SBCL) compiler v1.3.3 
- Frederic Peschanski's [`cl-jupyter`](https://github.com/fredokun/cl-jupyter)
- The [`hdf5-cffi`](https://github.com/HDFGroup/hdf5-cffi) Common Foreign Function Interface (CFFI) wrapper of the HDF5 library  
- [`alcove`](https://github.com/HDFGroup/alcove), a small collection of auxiliary Lisp functions developed for this blog  

Whenever we refer to 'the specification' in this document, we mean the [HDF5 File Format Specification](https://www.hdfgroup.org/HDF5/doc/H5.format.html).


## Reading the Superblock of an HDF5 File

As is common in many other file formats, every HDF5 file starts with a file format signature followed by a so-called [*superblock*](https://www.hdfgroup.org/HDF5/doc/H5.format.html#Superblock), a small piece of metadata with just enough information to get started. For simplicity, we'll create a bare-bones HDF5 file, but you should feel free to use any of your own HDF5 files for this exercise.

In [1]:
(defparameter *file-name* "superblock.h5")

*FILE-NAME*

It's easy to use the Lisp [Common Foreign Function Interface](https://common-lisp.net/project/cffi/) (CFFI) to call the HDF5 library, and to create an "empty" HDF5 file.

In [2]:
(let ((fcpl (h5pcreate +H5P-FILE-CREATE+)))
     ;; create and close the file
     (h5fclose (h5fcreate *file-name* +H5F-ACC-TRUNC+
                          +H5P-DEFAULT+ +H5P-DEFAULT+))
     (h5pclose fcpl))

0

Dropping to the byte level, Lisp's [`OPEN`](http://clhs.lisp.se/Body/f_open.htm) function lets us create a file stream that is connected to the HDF5 file.

In [3]:
(defparameter *input-stream*
    (open *file-name* :direction :input :element-type '(unsigned-byte 8)))

*INPUT-STREAM*

For the moment, we assume that the superblock is located at the beginning of the HDF5 file. (We'll discuss so-called *user blocks* in the next installment.) The function `READ-SUPERBLOCK`, defined in the `ALCOVE` package, can be used to read the HDF5 file's superblock, which returns an [association list](http://clhs.lisp.se/Body/26_glo_a.htm#association_list) ("a list of key/value pairs"). 

In [4]:
(documentation 'read-superblock 'function)

"--------------------------------------------------------------[function-doc]
READ-SUPERBLOCK
Args: (INPUT-STREAM)
Locate the HDF5 file format signature in INPUT-STREAM and read the superblock.
Return the superblock as an association list or NIL if the file format
signature is not found.
------------------------------------------------------------------------------"

We loop over the list returned by `READ-SUPERBLOCK`, printing each element. 

In [5]:
(loop for x in (read-superblock *input-stream*)
      do (format t "~a~%" x))

(SBLK-VERSION . 0)
(FREE-SPACE-VERSION . 0)
(ROOT-STBL-VERSION . 0)
(SOHM-VERSION . 0)
(SIZE-OF-OFFSETS . 8)
(SIZE-OF-LENGTHS . 8)
(GROUP-LEAF-NODE-K . 4)
(GROUP-INTERNAL-NODE-K . 16)
(FILE-CONSISTENCY-FLAGS . #(0 0 0 0))
(BASE-ADDRESS . 0)
(FREE-SPACE-INFO-ADDRESS . 18446744073709551615)
(EOF-ADDRESS . 800)
(DRIVER-INFO-BLOCK-ADDRESS . 18446744073709551615)
(ROOT-STBL-ENTRY (LINK-NAME-OFFSET . 0) (OHDR-ADDRESS . 96)
  (CACHE-TYPE . #(1 0 0 0)) (SCRATCH . #(136 0 0 0 0 0 0 0 168 2 0 0 0 0 0 0)))


NIL

The meaning of the different fields is decribed in the [HDF5 File Format Specification](https://www.hdfgroup.org/HDF5/doc/H5.format.html#Superblock).

18,446,744,073,709,551,615 is the *undefined address* $2^{64}-1$.

We defer a discussion of *versioning* in HDF5 to a future installment. The superblock has a version attribute attached to it, but, as we well see soon, almost all primitives decribed in the specification are versioned. It's a way of keeping an open mind about the future.

What we just saw was a version 0 superblock. In the next section, we'll look at a version 2 superblock. What about version 1? The specfication has this to say:

>Version 0 is the default format, while version 1 is basically the same as version 0 with additional information when a non-default B-tree 'K' value is stored. Version 2 is the latest format, with some fields eliminated or compressed and with superblock extension and checksum support.

In [6]:
(close *input-stream*)

T

### Superblock Version 2

In [7]:
(setf *file-name* "superblock.v2.h5")

"superblock.v2.h5"

We create an HDF5 file with a version 2 superblock by specifying a non-default *file access property list*, which instructs the HDF5 library to use the latest version for creating objects in the file.

In [8]:
(let ((fapl (h5pcreate +H5P-FILE-ACCESS+)))
     (h5pset-libver-bounds fapl :H5F-LIBVER-LATEST :H5F-LIBVER-LATEST)
     (h5fclose (h5fcreate *file-name* +H5F-ACC-TRUNC+ +H5P-DEFAULT+ fapl))
     (h5pclose fapl))

0

Read the superblock from a file stream as we did previously:

In [9]:
(setf *input-stream* (open *file-name* :direction :input :element-type '(unsigned-byte 8)))

#<SB-SYS:FD-STREAM for "file /notebooks/superblock.v2.h5" {10066AD043}>

In [10]:
(loop for x in (read-superblock *input-stream*)
      do (format t "~a~%" x))

(SBLK-VERSION . 2)
(SIZE-OF-OFFSETS . 8)
(SIZE-OF-LENGTHS . 8)
(FILE-CONSISTENCY-FLAGS . 0)
(BASE-ADDRESS . 0)
(SBLK-EXT-ADDRESS . 18446744073709551615)
(EOF-ADDRESS . 195)
(ROOT-OHDR-ADDRESS . 48)
(SBLK-CHECKSUM . #(171 95 237 54))


NIL

The version 2 superblock appears a little more clearly arranged. The two new ingredients are the superblock checksum and a slot for [superblock extensions](https://www.hdfgroup.org/HDF5/doc/H5.format.html#SuperblockExt).

>All checksums used in the format are computed with the [Jenkins lookup](http://www.burtleburtle.net/bob/hash/doobs.html) algorithm.

In [11]:
(close *input-stream*)

T

At the time of this writing, superblock version 3 is being finalized. It's practically identical to version 2, but stay tuned for an update.